In [ ]:
import pandas as pd
from backtesting import Backtest, Strategy


/opt/homebrew/Caskroom/miniconda/base/lib/python3.13/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

# Reading in data and setting up training and testing sets
- Here we have set up a 70 / 30 training / testing set data split
- The reason for setting up training and testing sets is so we can optimise parameters on the training set, and then test them on our testing set to see if the parameters work on unseen data

In [2]:
df = pd.read_csv('sol_5min_data.csv', parse_dates=True, index_col=0)
print(df)

#only use data from 2024 onwards
df = df[df.index > pd.Timestamp('12-01-2024')]

split_date = df.index[int(len(df) * 0.7)]

df_train = df.loc[:split_date]
df_test  = df.loc[split_date:]

print('Training set:')
print(df_train.describe())
print('Testing set:')
print(df_test.describe())





                       Open    High     Low   Close
timestamp                                          
2023-10-28 19:55:00   31.58   31.61   31.54   31.58
2023-10-28 20:00:00   31.58   31.64   31.55   31.64
2023-10-28 20:05:00   31.64   31.69   31.62   31.68
2023-10-28 20:10:00   31.68   31.71   31.59   31.60
2023-10-28 20:15:00   31.60   31.70   31.60   31.70
...                     ...     ...     ...     ...
2025-10-27 20:30:00  199.16  199.16  198.78  198.94
2025-10-27 20:35:00  198.93  199.06  198.66  198.94
2025-10-27 20:40:00  198.94  199.44  198.94  199.35
2025-10-27 20:45:00  199.35  199.56  199.14  199.40
2025-10-27 20:50:00  199.41  199.57  199.18  199.21

[210252 rows x 4 columns]
Training set:
               Open          High           Low         Close
count  66704.000000  66704.000000  66704.000000  66704.000000
mean     170.229807    170.540795    169.912455    170.228986
std       36.445961     36.543570     36.339658     36.445142
min       95.720000     97.640000  

# Defining our strategy and its parameters
- We define our strategy as `BollingerRSIStrategy` which is an instance of `Strategy` (a built in class)
- We define the parameters that our strategy will use
- Our strategy has to have these two functions: `init` and `next`
## `init` function
- init (initialise) is the function ran when the strategy is created
- we use this function to define the logic that our trading bot will use
- in this example we use it to precompute our bollinger bands and rsi for all the values in the dataset
- these precomputed values can then easily be used in the next function

## `next` function
- during the backtest, the next function will be ran on every single data point
- this is where we include the actual trade execution logic 
- here we will enter a position (buy) if rsi is signalling underbought and price is below the bottom bollinger band
- we will close the position (sell) when the inverse happens, rsi signals overbought and price is above the upper bollinger band
- NOTE: this is simple logic where we only buy and sell spot, no shorts, we always buy with maximum available capital
- I have added an extra parameter in the buy function, I set `sl=0.95*price`, meaning if drops 5% from where the position was entered, we execute our stop loss, closing the position to prevent further losses

In [3]:
class BollingerRSIStrategy(Strategy):
    bb_period = 20
    bb_std = 2
    rsi_period = 14
    rsi_lower = 30
    rsi_upper = 70



    def init(self):
        close = pd.Series(self.data.Close, index=self.data.index)
        sma = close.rolling(window=self.bb_period).mean()
        std = close.rolling(window=self.bb_period).std()

        #bb bands are defined as moving average +/- standard deviations (number of std can differ)
        bb_upper_data = sma + (std * self.bb_std)
        bb_lower_data = sma - (std * self.bb_std)
        
        self.bb_upper = self.I(lambda: bb_upper_data, name='BB_Upper')
        self.bb_middle = self.I(lambda: sma, name='BB_Middle')
        self.bb_lower = self.I(lambda: bb_lower_data, name='BB_Lower')
        
        delta = close.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=self.rsi_period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=self.rsi_period).mean()
        rs = gain / loss
        rsi_data = 100 - (100 / (1 + rs))
        self.rsi = self.I(lambda: rsi_data, name='RSI')

    def next(self):
        price = self.data.Close[-1] #get the current price at this point in time
        #skip if indicators are not ready - do this because indicators are calculated on a rolling basis e.g. bb requires 20 candles
        if pd.isna(self.rsi[-1]) or pd.isna(self.bb_lower[-1]):
            return
    
        if price >= self.bb_upper[-1] and self.rsi[-1] > self.rsi_upper:
            #if price goes above, and we are still long, close and short
            self.position.close()
        elif price <= self.bb_lower[-1] and self.rsi[-1] < self.rsi_lower:  
            #if price goes below, and we are still short, close and buy
            self.buy(sl=0.95*price)#stop loss at 5% below entry price


# Running a backtest on the strategy
## Backtest function parameter explanation:
- `df_train` - our dataset that we will run the strategy on
- `BollingerRSIStrategy` - the instance of Strategy that we defined, the trading logic to be backtested
- `cash` - how much money the backtest will begin with
- `exclusive_orders` - whether or not we can have multiple trades open at the same time, in this instance True means we cannot
- `finalize_trades` - at the end of the backtest any open trades will be closed at the final price in the dataset

## Displaying key performance metrics
- here we print the stats from the backtest
- the power of backtesting.py is these metrics, they are all calculated for us allowing us to easily evaluate a strategy across these metrics

In [4]:
bt_train = Backtest(df_train, BollingerRSIStrategy, cash=10000, exclusive_orders=True, finalize_trades=True)
stats = bt_train.run()
print(stats)

# Save the stats before optimization for later comparison
stats_before_opt = stats.copy()



Backtest.run:   0%|          | 0/66684 [00:00<?, ?bar/s]

Start                     2024-12-01 00:05:00
End                       2025-07-20 14:40:00
Duration                    231 days 14:35:00
Exposure Time [%]                    52.80793
Equity Final [$]                   10304.0065
Equity Peak [$]                    13468.4885
Return [%]                            3.04007
Buy & Hold Return [%]               -23.94449
Return (Ann.) [%]                     2.76617
Volatility (Ann.) [%]                72.90546
CAGR [%]                              4.83273
Sharpe Ratio                          0.03794
Sortino Ratio                         0.05975
Calmar Ratio                          0.05312
Alpha [%]                            16.85406
Beta                                  0.57692
Max. Drawdown [%]                   -52.07015
Avg. Drawdown [%]                    -1.80823
Max. Drawdown Duration      182 days 00:50:00
Avg. Drawdown Duration        2 days 11:15:00
# Trades                                 1928
Win Rate [%]                      

# Plotting the strategy to see a visualisation of the performance

In [ ]:
bt_train.plot()


/opt/homebrew/Caskroom/miniconda/base/lib/python3.13/site-packages/backtesting/_plotting.py:141: UserWarning: Data contains too many candlesticks to plot; downsampling to '1h'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


GridPlot(id='p1368', ...)

# Now to optimise the strategy
- Our strategy had parameters (e.g. `bb_period`, `rsi_period`, `rsi_upper`, etc.) we can optimise the strategy to find the best parameters
- We can optimise for whatever we would like, in this instance we optimise for Sharpe Ratio
- The optimise function will simply run through all possible combinations of parameters and find the one with the highest Sharpe Ratio
- This can be slow, so we can add constraints for parameter combinations that we will allow. e.g. `rsi_upper` cannot be lower than `rsi_lower`

In [6]:
def optimise(series):
    return series['Sharpe Ratio']


def constraint(series):
    rsi_period_no_overlap = series['rsi_upper'] > series['rsi_lower']
    bb_period_no_overlap = series['bb_period'] > series['rsi_period']

    return rsi_period_no_overlap and bb_period_no_overlap 


# Running the optimisation
- We provide ranges of values for each parameter that we want to be tested
- `bb_period=range(10,30,5)` will test all combinations of bb_period from 10 to 30 incrementing by 5 each time. That is 10, 15, 20, 25, 30
- as it is doing an exhaustive search, running backtests for all combinations, it can take a long time if we specify lots of values
- take note that we specified to maximize for our optimise function (`maximize=optimise`), so it will find the parameters that maximise for whatever our optimise function returns. 
- It is important to note that if we wanted to optimise to minimise a metric we would change this, e.g. minimise the maximum drawdown faced by the strategy

In [7]:
stats = bt_train.optimize(
bb_period=range(10, 30, 5), 
bb_std=[1, 1.5, 2, 2.5, 3], 
rsi_period=range(5,30,5), 
rsi_lower=range(10,30,5), 
rsi_upper=range(70,90,5), 
maximize=optimise, 
constraint=constraint)


/opt/homebrew/Caskroom/miniconda/base/lib/python3.13/site-packages/backtesting/backtesting.py:1624: UserWarning: Searching for best of 800 configurations.
  output = _optimize_grid()
/opt/homebrew/Caskroom/miniconda/base/lib/python3.13/site-packages/backtesting/backtesting.py:1624: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/800 [00:00<?, ?it/s]

Backtest.run:   0%|          | 0/66694 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/66694 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/66689 [00:00<?, ?bar/s]

...

Backtest.run:   0%|          | 0/66679 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/66679 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/66679 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/66679 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/66684 [00:00<?, ?bar/s]

In [8]:
print(stats)
stats._strategy

Start                     2024-12-01 00:05:00
End                       2025-07-20 14:40:00
Duration                    231 days 14:35:00
Exposure Time [%]                     5.41047
Equity Final [$]                     13446.98
Equity Peak [$]                      13506.26
Return [%]                            34.4698
Buy & Hold Return [%]               -23.94449
Return (Ann.) [%]                    59.35373
Volatility (Ann.) [%]                30.80864
CAGR [%]                             59.47957
Sharpe Ratio                          1.92653
Sortino Ratio                         5.29264
Calmar Ratio                          3.56802
Alpha [%]                            36.30975
Beta                                  0.07684
Max. Drawdown [%]                   -16.63494
Avg. Drawdown [%]                     -1.2988
Max. Drawdown Duration      134 days 05:00:00
Avg. Drawdown Duration        2 days 20:05:00
# Trades                                   67
Win Rate [%]                      

<Strategy BollingerRSIStrategy(bb_period=20,bb_std=2.0,rsi_period=15,rsi_lower=10,rsi_upper=70)>

# Now we will test the optimized parameters on the testing set
- Here, we pass in `df_test`, our testing dataset which contains unseen data.  
- We again call `.run()`, but this time we pass in our optimized parameters. This runs the backtest using those parameters rather than the ones defined in the Strategy class.  
- Finally, we print the performance statistics of this backtest to see how our optimized parameters performed on the testing set.  

## Did we overfit?
- We want to determine whether the optimized parameters were overfitted to the training set.  
- Overfitting occurs when a strategy learns patterns that are too specific to the training data, failing to capture the general, repeatable signals we are aiming for.  
- As a result, it performs well on the training data but fails to replicate that performance on unseen testing data.
- We can compare performance on training set and testing set to determine if the parameters were overfit


In [9]:
bt_test = Backtest(df_test, BollingerRSIStrategy, cash=10000, exclusive_orders=True, finalize_trades=True)

# run the backtest using the optimised parameters
optimised_strategy = stats._strategy
print(optimised_strategy)
test_stats = bt_test.run(
    **optimised_strategy._params
)

#printing all stats side by side for comparison
print(f"{'Metric':<30} | {'TRAINING (Before)':<25} | {'TRAINING (Optimized)':<25} | {'TEST SET':<25}")
print("-" * 115)

for metric in stats.index:
    if metric not in ['_strategy', '_equity_curve', '_trades']:
        train_before_val = stats_before_opt[metric]
        train_after_val = stats[metric]
        test_val = test_stats[metric]
        print(f"{metric:<30} | {str(train_before_val):<25} | {str(train_after_val):<25} | {str(test_val):<25}")



BollingerRSIStrategy(bb_period=20,bb_std=2.0,rsi_period=15,rsi_lower=10,rsi_upper=70)


Backtest.run:   0%|          | 0/28567 [00:00<?, ?bar/s]

Metric                         | TRAINING (Before)         | TRAINING (Optimized)      | TEST SET                 
-------------------------------------------------------------------------------------------------------------------
Start                          | 2024-12-01 00:05:00       | 2024-12-01 00:05:00       | 2025-07-20 14:40:00      
End                            | 2025-07-20 14:40:00       | 2025-07-20 14:40:00       | 2025-10-27 20:50:00      
Duration                       | 231 days 14:35:00         | 231 days 14:35:00         | 99 days 06:10:00         
Exposure Time [%]              | 52.80792756056608         | 5.410470136723435         | 4.425088326861861        
Equity Final [$]               | 10304.006500000018        | 13446.979999999985        | 11598.970000000001       
Equity Peak [$]                | 13468.488500000001        | 13506.259999999986        | 11854.45                 
Return [%]                     | 3.040065000000177         | 34.46979999999985 

# Plotting the optimised strategy on our test set to visualise its performance


In [12]:
bt_test.plot()


/opt/homebrew/Caskroom/miniconda/base/lib/python3.13/site-packages/backtesting/_plotting.py:141: UserWarning: Data contains too many candlesticks to plot; downsampling to '15min'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


GridPlot(id='p2551', ...)